In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
#from ExtractGenre import *
from CNN_ExtractGenre import *
from PolyphonicPreprocessing import *
import Util as Util

import DatasetLoader as DL



In [12]:
GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [17]:
Path = os.path.realpath('clean_midi/Eminem/Bad Meets Evil.mid')
mid = mido.MidiFile(Path)
#print(mid)

In [28]:
InputPath = os.path.relpath('clean_midi')

#Selecting a random number of directory
all_dirs = [d for d in os.listdir(InputPath) if os.path.isdir(os.path.join(InputPath, d))]

random_dirs = np.random.choice(all_dirs, 10)

for dir in tqdm(random_dirs, desc='Preprocessing'):
   DirPath = os.path.join(InputPath, dir)

   if not os.path.isdir(DirPath):
      continue

   #Real all the file in each folder
   for file in os.listdir(DirPath):

      FilePath = os.path.join(DirPath, file)

      #Cleaned monophonic: Some songs are corrupted:
      mid = mido.MidiFile(FilePath)
      for track in mid.tracks:
         for msg in track:
            if hasattr(msg, 'channel') and msg.channel == 10:
               #print(dir, file, msg)
               pass


Preprocessing: 100%|██████████| 10/10 [00:02<00:00,  4.10it/s]


In [29]:
test = PreProcessing(nDir = 30)

Preprocessing: 100%|██████████| 30/30 [00:03<00:00,  7.82it/s]


In [30]:
test['Percussion'][0]

{'SongName': ('My Number One', 'My Number One'),
 'Bars': (tensor(indices=tensor([[55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
                          55, 55, 58, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63,
                          63, 63, 63, 63, 63, 67],
                         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                          14, 15, 15,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
                          11, 12, 13, 14, 15, 15]]),
         values=tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
         size=(128, 16), nnz=34, dtype=torch.int32, layout=torch.sparse_coo),
  tensor(indices=tensor([[53, 55, 55, 55, 55, 55, 56, 56, 56, 56, 56, 56, 56, 58,
                          58, 58, 58, 58, 62, 63, 63, 63, 63, 63, 65, 65, 65, 65,
                          65, 65, 65, 67, 67, 67, 67, 67],
                         [15, 11, 12, 13, 14, 15,  5,  6, 

In [7]:
polconc = []
for i in range(30, 50):
   for j in range(2):
      polconc.append(test['Piano'][i]['Bars'][j].to_dense())

In [8]:
conc = np.concatenate(polconc, axis = 1)
MonoBarsToMIDI(conc, title = 'test', Instrument = 4)

In [9]:
poltest = PolyphonicPreProcessing(nDir = 30)

100%|██████████| 7/7 [00:00<00:00, 582.02it/s]


In [10]:
polconc = []
for i in range(30, 50):
   for j in range(2):
      polconc.append(poltest['rock'][i]['Bars'][j].to_dense())

In [11]:
conc = np.concatenate(polconc, axis = 2)
poltest['rock'][30]['Program']
print(conc.shape)

(4, 128, 640)


In [12]:
PolyBarsToMIDI(conc, title = 'poltest', Instrument = [113, 25, 1, 41])